In [1]:
import pandas as pd
import re
import numpy as np
import gc
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [2]:
df_contribuion = pd.read_csv('data/contribution_processed.csv')
df_companies = pd.read_csv('data/ma-companies-on-linkedin.csv', sep = ';')

In [3]:
companies_list = df_companies['Company name'].unique()
employer_list = df_contribuion['Employer'].unique()

In [4]:
def clean_text(text):
    '''
    Remove special charecters, numbers
    Return lower case alphabet charecters only
    '''
    text = str(text).lower()
    text = re.sub('[^A-Za-z]+', '', text)
    return text

companies_list_clean = np.array([clean_text(x) for x in companies_list.copy()])
employer_list_clean = np.array([clean_text(x) for x in employer_list.copy()])

In [7]:
sub_com = companies_list_clean[:100]
sub_emp = employer_list_clean[:100]

In [2]:
df_experiment = pd.read_csv('data/contribution_experiment.csv')

In [7]:
pd.set_option('display.max_rows', 500)
df_experiment['Industry_new'].value_counts()

law practice                            1890
hospital & health care                  1748
professional training & coaching        1733
government relations                    1190
government administration               1042
real estate                              949
higher education                         867
marketing and advertising                734
non-profit organization management       634
construction                             617
insurance                                524
financial services                       456
information technology and services      452
education management                     450
management consulting                    448
restaurants                              383
banking                                  363
public relations and communications      360
wholesale                                354
computer software                        337
food & beverages                         326
transportation/trucking/railroad         282
law enforc

In [97]:
industry_dict = {'Biotech/Pharma': ['biotechnology', 'pharmaceuticals'],
                'Education': ['higher education', 'education management', 'primary/secondary education'],
                'Healthcare':['hospital & health care', 'health, wellness and fitness', 'medical practice', 'mental health care', 'medical devices'],
                'Real Estate':['real estate', 'construction', 'commercial real estate', 'architecture & planning', 'facilities services', 'building materials'],
                'Law Practice':['law practice', 'legal services'],
                'Government Relations':['government relations'],
                'IT':['information technology and services', 'computer software', 'internet', 'semiconductors', 'telecommunications', 'computer & network security', 'graphic design'],
                'Government':['government administration', 'defense & space'],
                'Finance,Banking and Insurance':['financial services', 'banking', 'insurance', 'accounting', 'investment management', 'venture capital & private equity', 'investment banking'],
                'Accomodation and Food Services':['restaurants', 'hospitality', 'entertainment'],
                'Business Management': ['management consulting', 'public relations and communications'],
                'Retail and Wholesale Trade': ['wholesale', 'food & beverages', 'wine and spirits', 'retail', 'food production'],
                'Transportation': ['transportation/trucking/railroad', 'automotive', 'warehousing'],
                'Utilities': ['oil & energy', 'utilities']
                }

In [109]:
def industry_change(name):
    for k, v in industry_dict.items():
        if name in v:
            return k

In [116]:
df_experiment['Industry_new'] = df_experiment['Industry_new'].apply(industry_change)

In [119]:
df_experiment.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Address', 'Amount',
       'City', 'Contributor_ID', 'Contributor_Type', 'Date', 'Employer',
       'First_Name', 'Full_Name', 'ID', 'Last_Name', 'Line_Sequence',
       'Occupation', 'Principal_Officer', 'Report_ID', 'State', 'Zip',
       'Donor_Name', 'Industry', 'PAC', 'Lobbyist', 'Industry_new',
       'Industry_new1'],
      dtype='object')

In [121]:
df_experiment.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Industry_new1'], inplace=True)

In [124]:
df_experiment.to_csv('data/contribution_experiment1.csv')

In [125]:
df_experiment

,Address,Amount,City,Contributor_ID,Contributor_Type,Date,Employer,First_Name,Full_Name,ID,...,Occupation,Principal_Officer,Report_ID,State,Zip,Donor_Name,Industry,PAC,Lobbyist,Industry_new
0,270 S. Common Street,500.00,Lynn,NaN,Individual,2015-01-01,McGrath Enterprises,Patrick,Daniel F. Cahill,9166451,...,Developer,NaN,217037,MA,01905,PATRICK MCGRATH,NaN,NaN,False,None
1,186 Perkins Row,250.00,Topsfield,NaN,Individual,2015-01-15,Self,Thomas,Daniel F. Cahill,9166452,...,Attorney,NaN,217037,MA,01983,THOMAS DEMAKIS,Healthcare,NaN,False,None
2,11 Lake Ave.,25.00,Lynn,NaN,Individual,2015-03-02,NaN,Loretta,Daniel F. Cahill,9168914,...,NaN,NaN,217124,MA,10904,LORETTA O'DONNELL,NaN,NaN,False,None
3,7 William St.,50.00,Lynn,NaN,Individual,2015-03-02,NaN,Stephen,Daniel F. Cahill,9168915,...,NaN,NaN,217124,MA,01904,STEPHEN SPENCER,NaN,NaN,False,None
4,20 Bulfinch Ter.,50.00,Lynn,NaN,Individual,2015-03-02,NaN,Ann Marie,Daniel F. Cahill,9168916,...,NaN,NaN,217124,MA,01902,ANN MARIE LEONARD,NaN,NaN,False,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118456,45 Winthrop Street,50.00,Boston,NaN,Individual,2018-09-06,NaN,Np,Chynah Tyler,13404233,...,NaN,NaN,712794,MA,02119,NP RIGGS,NaN,NaN,False,None
118457,50 Forsyth Street,150.00,Boston,NaN,Individual,2018-09-06,Self,Chicken,Chynah Tyler,13404234,...,Resturant,NaN,712794,MA,02115,CHICKEN LOUS,Self-Employed,NaN,False,None
118458,Actblue,1413.42,Actblue,NaN,Individual,2018-08-28,Actblue,Actblue,Chynah Tyler,13404235,...,actable,NaN,712794,MA,NaN,ACTBLUE ACTBLUE,NaN,NaN,False,None
118459,60 Winthrop St,50.00,Roxbury,NaN,Individual,2018-09-06,NaN,Carl,Chynah Tyler,13404236,...,NaN,NaN,712794,MA,02119,CARL TODISCO,NaN,NaN,False,None
